<a href="https://colab.research.google.com/github/Abhis-123/Table-Detection/blob/main/Training_Yolo_For_Table_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
%cd "/content/drive/MyDrive"

/content/drive/MyDrive


In [51]:
!rm -r Table-Detection
!git clone https://github.com/Abhis-123/Table-Detection

rm: cannot remove 'Table-Detection': No such file or directory
Cloning into 'Table-Detection'...
remote: Enumerating objects: 1237, done.
remote: Counting objects: 100% (1237/1237), done.
remote: Compressing objects: 100% (828/828), done.
remote: Total 1237 (delta 404), reused 1219 (delta 389), pack-reused 0
Receiving objects: 100% (1237/1237), 91.74 MiB | 12.69 MiB/s, done.
Resolving deltas: 100% (404/404), done.
Checking out files: 100% (1147/1147), done.


In [53]:
%cd "Table-Detection/"

/content/drive/MyDrive/Table-Detection


In [54]:
%pip install -r requirements.txt # install dependencies

In [55]:
!python setup.py

100% 340/340 [00:02<00:00, 125.68it/s]
Missing: 0

yaml:
names:
- table
nc: 1
path: /content/drive/MyDrive/Table-Detection
train: /content/drive/MyDrive/Table-Detection/yolo_cache/train.txt
val: /content/drive/MyDrive/Table-Detection/yolo_cache/val.txt



In [56]:
!python train.py --img 512 --batch 16 --epochs 30 --data yolo.yaml --weights yolov5l.pt

train: weights=yolov5l.pt, cfg=, data=yolo.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
fatal: ambiguous argument 'main..origin/master': unknown revision or path not in the working tree.
Use '--' to separate paths from revisions, like this:
'git <command> [<revision>...] -- [<file>...]'
github: Command 'git rev-list main..origin/master --count' returned non-zero exit status 128.
YOLOv5 🚀 2021-12-3 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_ep

In [72]:
!python detect.py --weight runs/train/exp/weights/best.pt --source dataset/images/00018.PNG  

detect: weights=['runs/train/exp/weights/best.pt'], source=dataset/images/00018.PNG, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2021-12-3 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 367 layers, 46108278 parameters, 0 gradients, 107.8 GFLOPs
image 1/1 /content/drive/MyDrive/Table-Detection/dataset/images/00018.PNG: 640x512 3 tables, Done. (0.127s)
Speed: 0.7ms pre-process, 126.8ms inference, 2.7ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp9


In [73]:
import torch

In [74]:
def load_model(Best_Model, conf=0.25, iou=0.50):
    model = torch.hub.load('.',
                           'custom',
                           path=Best_Model,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model


def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
    if len(bboxes):
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        return bboxes, confs
    else:
        return [],[]
    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

In [76]:
model = load_model("runs/train/exp/weights/best.pt")

requirements: PyYAML==6.0 not found and is required by YOLOv5, attempting auto-update...

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 1373fa7 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)




requirements: 2 packages updated per /content/Table-Detection/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 367 layers, 46108278 parameters, 0 gradients, 107.8 GFLOPs
Adding AutoShape... 


In [77]:
from PIL import Image
import cv2 

In [84]:
img = cv2.imread("dataset/images/00019.PNG")
results =model(img , size=512)


In [86]:
results.print()

image 1/1: 951x735 1 table
Speed: 4.7ms pre-process, 104.5ms inference, 2.3ms NMS per image at shape (1, 3, 512, 416)


In [87]:
results.pred

[tensor([[ 42.21899, 379.37845, 701.83600, 754.57770,   0.82324,   0.00000]], device='cuda:0')]

In [88]:
type(results)

models.common.Detections

In [93]:
results.display(pprint=False, show=True, save=True, crop=False, render=False, save_dir=Path('.'))

Saved 1 image to .
